# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data

**Fecha**:21/02/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://873bad4e62fe:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 22:14:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Netflix service

In [3]:
from team_bellaco.spark_utils import SparkUtils
from pyspark.sql.functions import to_date
from importlib import reload
import team_bellaco.spark_utils
reload(team_bellaco.spark_utils)
netflix_schema = SparkUtils.generate_schema([
    ("show_id", "string"), 
    ("type", "string"),
    ("title", "string"),
    ("director", "string"),
    ("country", "string"),
    ("date_added", "string"),
    ("release_year", "integer"),
    ("rating", "string"),
    ("duration", "string"),
    ("listed_in", "string")
])

show_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

# Transformar la fecha con el formato correcto
show_df = show_df.withColumn("date_added", to_date(show_df["date_added"], "M/d/yyyy"))


show_df.printSchema()

show_df.show(5, truncate=False)


root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|2021-09-25|2020        |PG-13 |90 min  |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |2021-09-24|2021        |TV-MA |1 Season|Crime TV Shows, International TV Shows, TV Action & Adventure|
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|2021-

### Limpiar datos

In [4]:
# Limpiar el DataFrame según los requisitos
clean_show_df = SparkUtils.clean_df(show_df)


Number of nulls before cleaning:


+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       1|      1|         2|           2|     1|       2|        2|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+

Number of nulls after cleaning:


+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       0|      0|         0|           0|     0|       0|        0|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+

Filas originales: 8791, Filas después de limpieza: 8791


In [5]:
clean_show_df.show(5, truncate=False)

+-------+-------+--------------------------------+-----------------+----------------+----------+---------------------+---------------+-----------------+-------------------------------------------------------------+
|show_id|type   |title                           |director_non_null|country_non_null|date_added|release_year_non_null|rating_non_null|duration_non_null|listed_in_non_null                                           |
+-------+-------+--------------------------------+-----------------+----------------+----------+---------------------+---------------+-----------------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson  |United States   |2021-09-25|2020                 |PG-13          |90 min           |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq  |France          |2021-09-24|2021                 |TV-MA          |1 Seas

### Escribri el esquema

In [ ]:
SparkUtils.write_df(df=clean_show_df, path="/home/jovyan/notebooks/data/clean_netflix", format="parquet", mode="overwrite", partition_by="release_year")

Error al escribir el DataFrame: La columna de partición 'release_year' no existe en el DataFrame
Intentando escribir sin partición...


DataFrame escrito exitosamente en /home/jovyan/notebooks/data/clean_netflix en formato parquet (sin partición)


In [6]:
!ls notebooks/data/clean_netflix/ | wc -l

76


In [62]:
# Stop the SparkContext
sc.stop()